In [1]:
import pandas as pd
import contractions
import numpy as np
import nltk
import re

In [2]:
df = pd.read_csv('data.csv')
print("Shape:", df.shape)
df.head()

Shape: (2001, 22)


,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,...,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine / health,...,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,...,no,"english (fluently), tagalog (okay), french (po...",interested,fit,3.0,no,"i'm a straightforward, genuine, fun loving (i'...",painting,instruments,anywhere
2,fffe3300,Agnes Smith,20,seeing someone,f,bisexual,socially,sometimes,69.0,other,...,sometimes,"english (fluently), sign language (poorly), fr...",interested,fit,2.0,no,mmmmm yummy tacosss. yoga is where it's at. i ...,instruments,dancing,same city
3,fffe3400,Salvador Klaver,27,single,m,bisexual,socially,sometimes,68.0,computer / hardware / software,...,no,english,not interested,average,3.0,no,i'm a stealth geek. that special mix of techni...,sketching,acting,same city
4,fffe3500,Elana Sewell,22,single,f,bisexual,often,sometimes,68.0,other,...,yes,english,not interested,average,2.0,yes,with the whisper of the wind i was weaved into...,craft,designing,same city


In [3]:
df_copy = df[['age', 'height', 'education_level']]

In [4]:
df_copy.head()

,age,height,education_level
0,27,66.0,4.0
1,26,68.0,3.0
2,20,69.0,2.0
3,27,68.0,3.0
4,22,68.0,2.0


In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler = StandardScaler()
scaler.fit(df_copy['age'].values.reshape(-1, 1))
df_copy['age']=scaler.transform(df_copy['age'].values.reshape(-1, 1))

<ipython-input-6-d0af5a1cc543>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['age']=scaler.transform(df_copy['age'].values.reshape(-1, 1))


In [7]:
df_copy.head()

,age,height,education_level
0,-0.579402,66.0,4.0
1,-0.674817,68.0,3.0
2,-1.247305,69.0,2.0
3,-0.579402,68.0,3.0
4,-1.056475,68.0,2.0


In [8]:
scaler = StandardScaler()
scaler.fit(df_copy['height'].values.reshape(-1, 1))
df_copy['height']=scaler.transform(df_copy['height'].values.reshape(-1, 1))

<ipython-input-8-0149e4daae85>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['height']=scaler.transform(df_copy['height'].values.reshape(-1, 1))


In [9]:
scaler = StandardScaler()
scaler.fit(df_copy['education_level'].values.reshape(-1, 1))
df_copy['education_level']=scaler.transform(df_copy['education_level'].values.reshape(-1, 1))

<ipython-input-9-6ffeee925095>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['education_level']=scaler.transform(df_copy['education_level'].values.reshape(-1, 1))


In [10]:
df_copy.head()

,age,height,education_level
0,-0.579402,-0.588006,1.314342
1,-0.674817,-0.074431,0.083069
2,-1.247305,0.182356,-1.148204
3,-0.579402,-0.074431,0.083069
4,-1.056475,-0.074431,-1.148204


In [11]:
numeric_bio = np.array(df_copy)

In [12]:
def make_profile(row):
    user_id = row['user_id']
    age = row['age']
    status = row['status']
    sex = row['sex']
    orientation = row['orientation']
    drinks = row['drinks']
    drugs = row['drugs']
    smokes = row['smokes']
    height = row['height']
    job = row['job']
    city = row['location'].split(', ')[0]
    state = row['location'].split(', ')[1]
    location_preference = row['location_preference']
    pets = row['pets'].split(' and ')
    pet_likes = []
    pet_dislikes = []
    for i in pets:
        if 'likes' in i or 'has' in i:
            pet_likes.append(i.split(' ')[1])
        if 'dislikes' in i:
            pet_dislikes.append(i.split(' ')[1])
    languages = row['language'].split(', ')
    languages = [i.split()[0] for i in languages]
    languages = [i.split()[0] for i in languages]
    new_languages = row['new_languages']
    body_profile = row['body_profile']
    education = row['education_level']
    dropped_out = row['dropped_out']
    interests = [row['interests'], row['other_interests']]
    
    profile_dict = {'user_id': user_id,
                    'age': age,
                    'status': status,
                    'sex': sex,
                    'orientation': orientation,
                    'drinks': drinks,
                    'drugs': drugs,
                    'height': height,
                    'job': job,
                    'city': city,
                    'state': state,
                    'pet_likes': pet_likes,
                    'pet_dislikes': pet_dislikes,
                    'smokes': smokes,
                    'languages': languages,
                    'new_languages': new_languages,
                    'body_profile': body_profile,
                    'education': education,
                    'dropped_out': dropped_out,
                    'interests': interests,
                    'location_preference': location_preference}
    
    return profile_dict

In [13]:
user = []
for index, row in df.iterrows():
    user.append(make_profile(row))

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
numeric_sim = cosine_similarity(numeric_bio)

In [79]:
def check_compatibility(user_1, user_2):
    score = 0
    max_score = 0
    
    if user_1['user_id'] == user_2['user_id']:
        return(0, 140)

    
    # Status similarity score
    if user_1['status'] in ['single', 'available'] and user_2['status'] in ['single', 'available']:
        score += 10
    if user_1['status'] not in ['single', 'available'] and user_2['status'] not in ['single', 'available']:
        score += 10
    max_score += 10
    
    # Sexual Orientation Score
    if user_1['orientation'] == 'gay':
        if user_1['sex'] == 'f':
            like_1 = ['f', 'f']
        else:
            like_1 = ['m', 'm']
    if user_1['orientation'] == 'bisexual':
        like_1 = ['m', 'f']
    if user_1['orientation'] == 'straight':
        if user_1['sex'] == 'f':
            like_1 = ['m', 'm']
        else:
            like_1 = ['f', 'f']
    
    
    if user_2['orientation'] == 'gay':
        if user_2['sex'] == 'f':
            like_2 = ['f', 'f']
        else:
            like_2 = ['m', 'm']
    if user_2['orientation'] == 'bisexual':
        like_2 = ['m', 'f']
    if user_2['orientation'] == 'straight':
        if user_2['sex'] == 'f':
            like_2 = ['m', 'm']
        else:
            like_2 = ['f', 'f']
    
    if user_2['sex'] in like_1 and user_1['sex'] in like_2:
        score += 0
    else:
        return(0, 140)
    max_score += 0
    
    
    # Drinking score
    if user_1['drinks'] in ['rarely', 'not at all', 'socially'] and user_2['drinks'] in ['rarely', 'not at all', 'socially']:
        score += 10
    if user_1['drinks'] not in ['rarely', 'not at all', 'socially'] and user_2['drinks'] not in ['rarely', 'not at all', 'socially']:
        score += 10
    max_score += 10
    
    
    # Does drugs score
    if user_1['drugs'] == 'never' and user_2['drugs'] == 'never':
        score += 10
    if user_1['drugs'] == 'sometimes' and user_2['drugs'] == 'sometimes':
        score += 10
    if user_1['drugs'] == 'often' and user_2['drugs'] == 'often': 
        score += 10
    max_score += 10
    
    
    # Smoking score
    if user_1['smokes'] in ['no', 'trying to quit'] and user_2['smokes'] in ['no', 'trying to quit']:
        score += 10
    if user_1['smokes'] not in ['no', 'trying to quit'] and user_2['smokes'] not in ['no', 'trying to quit']:
        score += 10
    max_score += 10
    
    
    if user_1['job'] == user_2['job']:
        score += 10
    max_score += 10
    
    
    # Location score
    if user_1['location_preference'] == 'anywhere':
        city_like_1 = user_2['city']
        state_like_1 = user_2['state']
    if user_1['location_preference'] == 'same city':
        city_like_1 = user_1['city']
        state_like_1 = user_1['state']
    if user_1['location_preference'] == 'same state':
        city_like_1 = ''
        state_like_1 = user_1['state']
        
    if user_2['location_preference'] == 'anywhere':
        city_like_2 = user_1['city']
        state_like_2 = user_1['state']
    if user_2['location_preference'] == 'same city':
        city_like_2 = user_2['city']
        state_like_2 = user_2['state']
    if user_2['location_preference'] == 'same state':
        city_like_2 = ''
        state_like_2 = user_2['state']
        
    if (city_like_1 == user_2['city'] or state_like_1 == user_2['state']) and (city_like_2 == user_1['city'] or state_like_2 == user_1['state']):
        score += 20
    max_score += 20
    

    
    # Pet like/dislike score
    pet_like_1 = set(user_1['pet_likes'])
    pet_like_2 = set(user_2['pet_likes'])
    
    pet_dislike_1 = set(user_1['pet_dislikes'])
    pet_dislike_2 = set(user_2['pet_dislikes'])
    
    if len(pet_like_1.intersection(pet_dislike_2)) > 0 or len(pet_like_2.intersection(pet_dislike_1)) > 0:
        score += 0
    else:
        if len(pet_like_1.intersection(pet_like_2)) >= 1:
            score += 10
        if len(pet_dislike_1.intersection(pet_dislike_2)) >= 1:
            score += 10
    max_score += 20
    
    
    # Open to language score
    lang_1 = set(user_1['languages'])
    lang_2 = set(user_2['languages'])
    if len(lang_1.intersection(lang_2)) > 0:
        score += 20
    else:
        if user_1['new_languages'] != 'not interested' and user_2['new_languages'] != 'not interested':
            score += 20
    max_score += 20

    
    # Dropped out
    if user_1['dropped_out'] == user_2['dropped_out']:
        score += 5
    max_score += 5
    
    
    # Body type score
    if user_1['body_profile'] == user_2['body_profile']:
        score += 5
    max_score += 5
    
    
    # Interests score
    interests_1 = set(user_1['interests'])
    interests_2 = set(user_2['interests'])
    if len(interests_1.intersection(interests_2)) >= 1:
        score += 20
    max_score += 20
    
    return(score, max_score)

In [80]:
ns = 78

In [81]:
final = np.zeros((2001,2001))

for i in range(len(user)):
    for j in range(len(user)):
        score, max_score = check_compatibility(user[i], user[j])
        if score != 0:
            score += ns*numeric_sim[i][j]
            
        max_score += ns
        final[i][j] = max(0, 100*(score/max_score))

In [82]:
final

array([[ 0.        ,  0.        , 26.9085661 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [26.9085661 ,  0.        ,  0.        , ..., 45.46574995,
        41.58066079,  0.        ],
       ...,
       [ 0.        ,  0.        , 45.46574995, ...,  0.        ,
         0.        , 23.0246814 ],
       [ 0.        ,  0.        , 41.58066079, ...,  0.        ,
         0.        , 14.23304896],
       [ 0.        ,  0.        ,  0.        , ..., 23.0246814 ,
        14.23304896,  0.        ]])

In [83]:
index_values = list(df['user_id'])
column_values = list(df['user_id'])
df_to_submit = pd.DataFrame(data = final,  
                  index = index_values,  
                  columns = column_values)

In [84]:
df_to_submit

,fffe3100,fffe3200,fffe3300,fffe3400,fffe3500,fffe3600,fffe3700,fffe3800,fffe3900,fffe31003000,...,fffe3100390039003200,fffe3100390039003300,fffe3100390039003400,fffe3100390039003500,fffe3100390039003600,fffe3100390039003700,fffe3100390039003800,fffe3100390039003900,fffe3200300030003000,fffe3200300030003100
fffe3100,0.000000,0.0000,26.908566,0.000000,14.930873,0.000000,47.793886,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.901230,0.000000,0.000000,0.000000
fffe3200,0.000000,0.0000,0.000000,74.758200,0.000000,72.171196,0.000000,0.000000,74.747893,0.000000,...,20.158716,0.000000,60.951083,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fffe3300,26.908566,0.0000,0.000000,56.230328,71.927002,66.926863,20.070937,12.060030,0.000000,31.799693,...,0.000000,38.074132,0.000000,0.000000,12.112980,69.602141,64.858204,45.465750,41.580661,0.000000
fffe3400,0.000000,74.7582,56.230328,0.000000,54.678788,62.646230,0.000000,0.000000,70.114662,0.000000,...,9.271424,0.000000,65.089658,25.604749,0.000000,0.000000,0.000000,0.000000,0.000000,17.167247
fffe3500,14.930873,0.0000,71.927002,54.678788,0.000000,66.316914,23.549912,1.408274,0.000000,31.881718,...,0.000000,29.805755,0.000000,0.000000,1.639982,78.649225,63.263413,37.315594,30.287225,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe3100390039003700,0.000000,0.0000,69.602141,0.000000,78.649225,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.250807,0.000000,0.000000,0.000000,0.000000,0.000000,49.518835
fffe3100390039003800,31.901230,0.0000,64.858204,0.000000,63.263413,0.000000,30.924002,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fffe3100390039003900,0.000000,0.0000,45.465750,0.000000,37.315594,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,10.203487,0.000000,0.000000,0.000000,0.000000,0.000000,23.024681
fffe3200300030003000,0.000000,0.0000,41.580661,0.000000,30.287225,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,49.696072,0.000000,0.000000,0.000000,0.000000,0.000000,14.233049


In [85]:
df_to_submit.to_csv('numeric_submission_new.csv')